<a href="https://colab.research.google.com/github/sime1/notebooks/blob/master/perceptron.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Perceptron

Questo notebook contiene un'implementazione "didattica" dell'algoritmo **perceptron** creata utilizzando *NumPy*.

Ho implementato un perceptron binario (output -1 e +1), creando una classe che assomigli a quelle dei modelli di `sklearn`.

## Perceptron

La classe `Perceptron` permette di essere inizializzata con un `random_state`, e fornisce i metodi `fit`, `score` e `predict` per l'allenamento e la valutazione del modello.

### `fit`

La funzione `fit` inizializza casualmente il vettore $w$ dei pesi e poi itera
`max_passes` volte il dataset, aggiornando i pesi in base al valore di `rate`.
Durante il processo di training viene tenuta traccia del miglior valore di $w$, per poi farne uso dopo il training nei metodi `predict` e `score`

Ho utilizzato il pacchetto `tqdm` per visualizzare una progress bar durante il training.

### `score`

Come score viene utilizzata l'accuratezza media, ossia 

$1/n \sum_{i=0}^{n-1}1(o_{i}=t_{i})$

dove $o$ è il vettore contenente le label che prevede il modello, $t$ è il vettore contenente i valori noti delle label e "$=$" è l'operatore booleano che ritorna 1 se i due operandi sono uguali e 0 altrimenti.

In [0]:
import numpy as np
from tqdm import trange
from numpy import sign

class Perceptron:
  def __init__(self, random_state=None, max_passes=100, rate=0.0001):
    self.rate = rate
    self.rng = np.random.RandomState(seed=random_state)
    self.max_passes = max_passes
    self.rate = rate
    self.best_score = 0

  def __perceptron_pass(self, x,y):
    for i, p in enumerate(x):
      # aggiungo 1 per la costante b
      p = np.append([1], p)
      t = y[i]
      o = self.__output(p)
      if o != t:
        self.w = self.w + self.rate * (t - o) * p

  def __output(self, p):
    return sign(np.dot(p, self.w))

  def score(self, x, y):
    correct = 0
    for i, p in enumerate(x):
      p = np.append([1], p)
      if self.__output(p) == y[i]:
        correct += 1
    return correct / len(y)
  
  def fit(self, x, y):
    # + 1 perchè un peso corrisponde alla costante b (traslazione della retta)
    self.w = self.rng.random_sample(x.shape[1] + 1)
    with trange(self.max_passes) as t:
      for i in t:
        self.__perceptron_pass(x,y)
        score = self.score(x,y)
        t.set_description(f"pass {i}")
        t.set_postfix(score=score)
        if score > self.best_score:
          self.best_score = score
          self.best_w = self.w
        if score == 100:
          break
    self.w = self.best_w
  
  def predict(self,x):
    y = []
    for p in x:
      y.append(self.__output(p))
    return np.array(y)

## Breast Cancer Dataset

Definito il modello, ho provato ad utilizzarlo per la classificazione del dataset *breast cancer* fornito da `sklearn`

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_breast_cancer

dataset = load_breast_cancer()
x = dataset.data
# devo preprocessare i dati perché le classi siano +1 e -1
y = [ 1 if y == 1 else -1 for y in dataset.target ]
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.2,random_state=1)

model = Perceptron(random_state=1, max_passes=1000, rate=0.000001)
model.fit(x_train,y_train)
score = model.score(x_test, y_test)

print(f"\nscore: {score}")


pass 999: 100%|██████████| 1000/1000 [00:14<00:00, 71.43it/s, score=0.921]


score: 0.9122807017543859


## Osservazioni

* Come criterio di stop è stato implementato unicamente il numero massimo di passi. In un esempio più completo probabilmente verrebbe presa in considerazione anche la variazione di performance del modello, per fermarsi dopo un numero di passi nei quali il modello non è migliorato.
* Probabilmente sarebbe meglio usare il modello che ha migliore score sui dati di validation e non training. In tal caso bisognerebbe modificare la classe per poter ricevere in input anche i dati da usare per la validazione.
* Il modello è stato scritto in modo che fosse chiaro il funzionamento dell'algoritmo. In questo senso è interessante a scopo didattico, ma non è utilizzabile in uno scenario reale, dove metodi implementati nativamente e ottimizzati possono fornire prestazioni migliori di diversi ordini di grandezza.
